In [2]:
from translate import Translator
translator = Translator(to_lang="uk")
translation = translator.translate("Hello, world!")
print(translation)  # Outputs: 'Hola, mundo!'

Вітаю, світе!


In [1]:
import pandas as pd

In [ ]:
semanticforce_file = open('kyiv_digital_sentiment_annotation - Annotator X.tsv')
data = pd.read_csv(semanticforce_file,sep='\t')

In [4]:
data4 = pd.read_csv('telegram_train.csv')
data5 = pd.read_csv('telegram_validation.csv')

In [5]:
with open('emotions_sentiment_youscan - Annotator 1 - Veronika.tsv','r') as file3:
    data3=list(file3)
    data3=[e.replace('\n','').split('\t')[0:2] for e in data3[1:652]]
data2 = pd.DataFrame(data3, columns=['content', 'label'])


In [2]:
data6 = pd.read_csv('./data/truw.tsv',sep='\t')

In [3]:
print(len(data6))

276309


In [5]:
data6['channel_name'].value_counts()

channel_name
rian_ru              79663
ROSSIYA_SEGODNIA     69238
uniannet             67727
radiosvoboda         33220
UkrPravdaMainNews    22860
ZE_kartel             3601
Name: count, dtype: int64

In [10]:
data6['channel_attitude'].value_counts()

channel_attitude
pro-ru    152502
pro-ua    123807
Name: count, dtype: int64

In [15]:
data6['gpt_sentiment'].value_counts()

gpt_sentiment
negative    146808
neutral      92331
positive     37170
Name: count, dtype: int64

In [6]:
data6.head()

,Unnamed: 0,message_id,Date,text,channel_name,channel_attitude,gpt_geopolitical_attitude_standardized,gpt_sentiment,pymorphy_sentiment,gpt_hate_or_discrimination
0,0,0,2022-01-01 02:30:20+00:00,Группа бывших американских дипломатов и сотруд...,rian_ru,pro-ru,pro-ukrainian,neutral,negative,no
1,1,1,2022-01-02 22:08:02+00:00,Байден поговорил с Зеленским. Главные тезисы: ...,rian_ru,pro-ru,pro-ukrainian,negative,negative,no
2,2,2,2022-01-02 22:30:58+00:00,"Зеленский сообщил, что обсудил с Байденом дейс...",rian_ru,pro-ru,pro-ukrainian,negative,negative,no
3,3,3,2022-01-03 17:26:38+00:00,"В Вашингтоне сильнейшая метель. Байдену, котор...",rian_ru,pro-ru,not applicable,neutral,negative,no
4,4,4,2022-01-03 22:29:35+00:00,Захарова назвала отъезд Цимбалюка из России си...,rian_ru,pro-ru,pro-ukrainian,negative,negative,yes


In [9]:
data6.loc[4, 'text']

'Захарова назвала отъезд Цимбалюка из России синдромом Бабченко-Гордона. Много лет, живя в Москве, кричать про российско-украинскую войну, не вылезать из московских ресторанов, выпускать провокационно-фейковые помои в интернете, на все вопросы Чего же ты мучаешься в тылу врага и не пора ли вернуться к своим на передовую? отвечать бессвязным мычанием, а потом придумать байку об угрозе безопасности и статусе заложника. Сопоставимо со свиной кровью Аркадия Бабченко и спецоперациями-коллаборациями Дмитрия Гордона'

In [14]:
data6.loc[data6['channel_attitude'] == 'pro-ua', 'channel_name'].value_counts()
# data6.loc[3299, 'text']

channel_name
uniannet             67727
radiosvoboda         33220
UkrPravdaMainNews    22860
Name: count, dtype: int64

In [8]:
import langid
data6=data6[data6['channel_attitude']=='pro-ua']

# Define a function to detect the language
def detect_language(text):
    lang, _ = langid.classify(text)
    return lang

# Apply the function to the 'text' column
data6['language'] = data6['text'].apply(detect_language)

# Filter for Russian (ru) and Ukrainian (uk)
df_russian = data6[data6['language'] == 'ru']
df_ukrainian = data6[data6['language'] == 'uk']

print("Russian texts:")
print(len(df_russian))

print("\nUkrainian texts:")
print(len(df_ukrainian))

Russian texts:
121346

Ukrainian texts:
1752


In [9]:
df_ukrainian_p=df_ukrainian[df_ukrainian['gpt_sentiment']=='positive']
df_ukrainian_neu=df_ukrainian[df_ukrainian['gpt_sentiment']=='neutral']
df_ukrainian_neg=df_ukrainian[df_ukrainian['gpt_sentiment']=='negative']
selected_columns = [df[['text', 'gpt_sentiment']] for df in [df_ukrainian_neg,df_ukrainian_neu,df_ukrainian_p]]

# Concatenate the DataFrames
data6_uk = pd.concat(selected_columns, ignore_index=True)
data6_uk = data6_uk.rename(columns={'gpt_sentiment': 'label'})

data6_uk = data6_uk.rename(columns={'text': 'content'})

data6_uk['label'] = data6_uk['label'].str.capitalize()


In [15]:
len(data6_uk )

1752

In [16]:
len(df_ukrainian_neg),len(df_ukrainian_neu),len(df_ukrainian_p)

(469, 620, 663)

In [10]:
seed=42
df_russian_p=df_russian[df_russian['gpt_sentiment']=='positive'].sample(n=542,random_state=seed)
df_russian_neu=df_russian[df_russian['gpt_sentiment']=='neutral'].sample(n=2220,random_state=seed)
df_russian_neg=df_russian[df_russian['gpt_sentiment']=='negative'].sample(n=1180,random_state=seed)

In [11]:
selected_columns = [df[['text', 'gpt_sentiment']] for df in [df_russian_neg,df_russian_neu,df_russian_p]]

# Concatenate the DataFrames
data6_ru = pd.concat(selected_columns, ignore_index=True)
data6_ru = data6_ru.rename(columns={'gpt_sentiment': 'label'})

data6_ru = data6_ru.rename(columns={'text': 'content'})

data6_ru['label'] = data6_ru['label'].str.capitalize()


In [19]:
from deep_translator import GoogleTranslator

# Initialize the translator
translator = GoogleTranslator(source='russian', target='ukrainian')

# Function to translate text
def translate_text(text):
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"Error translating text: {text[:30]}... - {e}")
        return None

# Apply the translation to the 'content' column and create a new 'content_translated' column
data6_ru['content_translated'] = data6_ru['content'].apply(translate_text)

# Display the first few rows to verify the translation
print(data6_ru[['content', 'content_translated']].head())

Error translating text: Соледар - это такая абсолютная... - Request exception can happen due to an api connection error. Please check your connection and try again
Error translating text: Главное за день: - Суд в Босто... - Request exception can happen due to an api connection error. Please check your connection and try again
                                             content  \
0  Военные почти год строили дом своего начальник...   
1  День Победы из мемориального события и факта л...   
2  США против общего запрета на выдачу виз россия...   
3  В Швеции убили критика Кадырова, на которого у...   
4   Войска рф скинули взрывчатку на людей, которы...   

                                  content_translated  
0  Військові майже рік будували будинок свого нач...  
1  День Перемоги з меморіальної події та факту ос...  
2  США проти загальної заборони видачу віз росіян...  
3  У Швеції вбили критика Кадирова, на якого вже ...  
4  Війська Росії скинули вибухівку на людей, які ...  


In [21]:
data6_ru_t= data6_ru[['content_translated', 'label']].rename(columns={'content_translated': 'content'})

In [26]:
data6_ru_t.to_csv('./translated_russian.tsv', sep='\t',index=False)

In [22]:
df = pd.concat([data, data2,data4,data5,data6_uk,data6_ru_t], ignore_index=True)
print(df)

                                                content     label
0                      Шо по відключень енергії в Києві  Negative
1     Збудували фізичний захист для енергообʼєктів К...  Positive
2     Так я ж тебе задал вопрос. Киев, май, первое п...  Positive
3                                                Аварії  Negative
4     У вас давно вже щось відпадає,а ви ще до цих п...  Negative
...                                                 ...       ...
9903  Ще кадри зустрічі Зеленського із Дудою Разом в...  Positive
9904  Глава Луганської ОВА: Ситуація почне змінювати...  Positive
9905  Усі готові поставити Росію на місце. І після т...  Positive
9906  Герой мемов Чіткий паца з Рівного одружився Ав...  Positive
9907  Про це сказав прем'єр-міністр Нідерландів Марк...  Positive

[9908 rows x 2 columns]


In [23]:
label_mapping = {1:2,0:1,'Neutral': 0, 'Negative': 1, 'Positive': 2,'Very Positive':2,'Very Negative':1, 'Mixed':1}
df['label'] =df['label'].map(label_mapping)
df = df.dropna()
df['label'] =df['label'].astype(int)

In [24]:
labels=df['label'].values.tolist()
labels.count(0),labels.count(1),labels.count(2)

(3123, 3119, 3663)

In [27]:
set(labels)

{0, 1, 2}

In [28]:
import pandas as pd
from datasets import Dataset, load_metric
from sklearn.metrics import roc_auc_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split


/Users/nikahelicopter/miniconda3/envs/nika/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name ="seara/rubert-base-cased-russian-sentiment"
#model_name = "cointegrated/rubert-tiny-sentiment-balanced" 
#model_name = "dardem/xlm-roberta-large-uk-toxicity"
# You can choose a different model if needed
#sentiment = pipeline('text-classification',
    #model="seara/rubert-base-cased-russian-sentiment", 
    #return_all_scores=True
#)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/Users/nikahelicopter/miniconda3/envs/nika/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)

# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [31]:
from datasets import Dataset
from transformers import DataCollatorWithPadding

def tokenize_function(examples):
    return tokenizer(examples['content'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Map: 100%|█████████| 1486/1486 [00:00<00:00, 20340.03 examples/s]


In [32]:
from transformers import Trainer, TrainingArguments
import numpy as np

In [59]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=1e-5,  # Reduced learning rate
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # Increased number of epochs
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Monitor eval_loss to find the best model
    report_to=[]
)

# Define compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    # Apply softmax to get probabilities
    probs = np.exp(p.predictions) / np.exp(p.predictions).sum(-1, keepdims=True)
    
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    mcc = matthews_corrcoef(labels, preds)
    auroc = roc_auc_score(labels, probs, multi_class='ovr', average='weighted')
    return {
        'precision': precision['precision'],
        'recall': recall['recall'],
        'f1': f1,
        'mcc': mcc,
        'auroc': auroc,
    }

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results
print(eval_results)

# Save the model
model.save_pretrained('./fine_tuned_model5_tuwr_ukr_ru')
tokenizer.save_pretrained('./fine_tuned_model5_tuwr_ukr_ru')

/Users/nikahelicopter/miniconda3/envs/nika/lib/python3.9/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 8.61 GB, other allocations: 9.34 GB, max allowed: 18.13 GB). Tried to allocate 350.24 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [33]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score
from datasets import load_metric
import torch

# Ensure the dataset contains the correct columns
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# Define compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    # Apply softmax to get probabilities
    probs = np.exp(p.predictions) / np.exp(p.predictions).sum(-1, keepdims=True)

    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    mcc = matthews_corrcoef(labels, preds)
    auroc = roc_auc_score(labels, probs, multi_class='ovr', average='weighted')
    return {
        'precision': precision['precision'],
        'recall': recall['recall'],
        'f1': f1,
        'mcc': mcc,
        'auroc': auroc,
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=8,  # Reduce number of epochs
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to=[],
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    no_cuda=True,  # Force training on CPU
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Function to clear memory after evaluation
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        # For MPS (Apple Silicon) backend
        import gc
        gc.collect()
        torch.mps.empty_cache()

# Train the model
trainer.train()
clear_memory()  # Clear memory manually if needed

# Evaluate the model
eval_results = trainer.evaluate()
clear_memory()  # Clear memory after evaluation

# Print the evaluation results
print(eval_results)

# Save the model
model.save_pretrained('./fine_tuned_model6_transl')
tokenizer.save_pretrained('./fine_tuned_model6_transl')


/Users/nikahelicopter/miniconda3/envs/nika/lib/python3.9/site-packages/transformers/training_args.py:1331: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/Users/nikahelicopter/miniconda3/envs/nika/lib/python3.9/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Mcc,Auroc
0,0.894900,0.789165,0.658359,0.652759,0.653390,0.480374,0.826137
2,0.518200,0.950850,0.658617,0.656797,0.656154,0.484391,0.828755
4,0.380300,1.909476,0.644619,0.642665,0.641334,0.462376,0.812491
6,0.029800,2.285976,0.653293,0.651413,0.651252,0.476245,0.804328
7,0.000500,2.390991,0.650811,0.650740,0.650216,0.474282,0.803028


/var/folders/vf/tvpnk3c156l650kh37r9m9yw0000gq/T/ipykernel_16665/2639112635.py:19: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  precision_metric = load_metric("precision")
Checkpoint destination directory ./results/checkpoint-1052 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2105 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3157 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-4210 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-5262 already exists and is non-empty.S

{'eval_loss': 0.7891645431518555, 'eval_precision': 0.6583590330924717, 'eval_recall': 0.6527590847913862, 'eval_f1': 0.653389820987069, 'eval_mcc': 0.4803736162256301, 'eval_auroc': 0.8261368400844703, 'eval_runtime': 90.5158, 'eval_samples_per_second': 16.417, 'eval_steps_per_second': 4.11, 'epoch': 8.0}


('./fine_tuned_model6_transl/tokenizer_config.json',
 './fine_tuned_model6_transl/special_tokens_map.json',
 './fine_tuned_model6_transl/vocab.txt',
 './fine_tuned_model6_transl/added_tokens.json',
 './fine_tuned_model6_transl/tokenizer.json')

In [ ]:

cointegrated
/
rubert-tiny-sentiment-balanced 



YaraKyrychenko
/
ukraine-war-pov


dardem
/
xlm-roberta-large-uk-toxicity